In [1]:
from hive_metastore_client.builders import DatabaseBuilder
from hive_metastore_client import HiveMetastoreClient

from hive_metastore_client.builders import (
    ColumnBuilder,
    SerDeInfoBuilder,
    StorageDescriptorBuilder,
    TableBuilder,
)


HIVE_HOST = "hive-metastore"
HIVE_PORT = 9083

In [3]:
# Creating test database
database = DatabaseBuilder("test_db").build()

with HiveMetastoreClient(HIVE_HOST, HIVE_PORT) as hive_client:
    hive_client.create_database(database)

In [2]:
# Check list of databases
with HiveMetastoreClient(HIVE_HOST, HIVE_PORT) as hive_client:
    databases = hive_client.get_all_databases()
    for db in databases:
        print(f" - {db}")

 - default
 - test_db


In [10]:
# Create test Ozone table
columns = [
    ColumnBuilder("id", "int", "test id").build(),
    ColumnBuilder("descr", "string", "test attribute").build(),
]

partition_key = [
    ColumnBuilder("report_dt", "string", "test partition column").build(),
]

serde_info = SerDeInfoBuilder(
    serialization_lib="org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe"
).build()

storage_descriptor = StorageDescriptorBuilder(
    columns=columns,
    location="s3a://ozone-om/s3v/mybucket/test_db/test_table",
    input_format="org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat",
    output_format="org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat",
    serde_info=serde_info,
).build()

table = TableBuilder(
    table_name="test_table",
    db_name="test_db",
    owner="MegaSuper_DE",
    storage_descriptor=storage_descriptor,
    partition_keys=partition_key,
    table_type="EXTERNAL_TABLE",
    parameters={"EXTERNAL": "TRUE"},
).build()

In [11]:
with HiveMetastoreClient(HIVE_HOST, HIVE_PORT) as hive_metastore_client:
    hive_metastore_client.create_external_table(table)

MetaException: MetaException(message='java.lang.NumberFormatException: For input string: "60s"')

In [8]:
# Check if table exists
with HiveMetastoreClient(HIVE_HOST, HIVE_PORT) as hive_client:
    tables = hive_client.get_all_tables("test_db")
    for table in tables:
        print(f"  - {table}")

  - test_table
